# 1. Данные


Подключаем библиотеки


In [ ]:
import shutil #Библиотека для обработки файлов, групп файлов, и папок
import os #Библиотека для работы файловой системы
import zipfile #Библиотека для работы zip-файлов
from google.colab import drive #Модуль для работы Google Disk
from PIL import Image #Модуль для работы с изображениями

Получаем  ссылку для аутентификации Google-Disk


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Находим нужный архив с датасетом и извлекаем все

In [ ]:
zip_file = '/content/drive/MyDrive/CNN_mango/mango_dataset.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

In [ ]:
# Каталог с набором данных
data_dir = '/content/mango_dataset'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для проверки
val_data_portion = 0.2
# Часть набора данных для тестирования
test_data_portion = 0.1
# Количество элементов данных в одном классе
nb_images = 500

Функция для создания train, test, val директорий

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "Anthracnose"))
    os.makedirs(os.path.join(dir_name, "Bacterial Canker"))
    os.makedirs(os.path.join(dir_name, "Cutting Weevil"))
    os.makedirs(os.path.join(dir_name, "Die Back"))
    os.makedirs(os.path.join(dir_name, "Gall Midge"))
    os.makedirs(os.path.join(dir_name, "Healthy"))
    os.makedirs(os.path.join(dir_name, "Powdery Mildew"))
    os.makedirs(os.path.join(dir_name, "Sooty Mould"))

In [ ]:
create_directory(train_dir) # Создание train-директории
create_directory(val_dir) # Создание val-директории
create_directory(test_dir)

Функция для копирования данных в нужную директорию

In [ ]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index+1, end_index+1):
        shutil.copy2(os.path.join(source_dir ,  "Anthracnose (" + str(i) + ").jpg"),
                    os.path.join(dest_dir, "Anthracnose")) #1

        shutil.copy2(os.path.join(source_dir ,  "Bacterial Canker (" + str(i) + ").jpg"),
                   os.path.join(dest_dir, "Bacterial Canker")) #2

        shutil.copy2(os.path.join(source_dir ,  "Cutting Weevil (" + str(i) + ").jpg"),
                   os.path.join(dest_dir, "Cutting Weevil")) #3

        shutil.copy2(os.path.join(source_dir ,  "Die Back (" + str(i) + ").jpg"),
                   os.path.join(dest_dir, "Die Back")) #4

        shutil.copy2(os.path.join(source_dir,  "Gall Midge (" + str(i) + ").jpg"),
                   os.path.join(dest_dir , "Gall Midge")) #5

        shutil.copy2(os.path.join(source_dir,  "Healthy (" + str(i) + ").jpg"),
                   os.path.join(dest_dir , "Healthy")) #6

        shutil.copy2(os.path.join(source_dir,  "Powdery Mildew (" + str(i) + ").jpg"),
                   os.path.join(dest_dir , "Powdery Mildew")) #7

        shutil.copy2(os.path.join(source_dir ,  "Sooty Mould (" + str(i) + ").jpg"),
                   os.path.join(dest_dir, "Sooty Mould")) #8

Высчитываем вспомогательные данные

In [ ]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)

350
450


In [ ]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

# 2. Создание модели InceptionV3. Загрузка изображений в модель. Сохрание изображений.

> Блок с отступами



In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD


import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt

In [ ]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 224, 224
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 2800
# Количество изображений для проверки
nb_validation_samples = 800
# Количество изображений для тестирования
nb_test_samples = 400


Создание предобученной сети InceptionV3


In [ ]:
InceptionV3_model=InceptionV3(include_top=False,
                      weights='imagenet',
                      input_shape=((224, 224, 3)))

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255) #Создание генератора

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes = ['Anthracnose','Bacterial Canker','Cutting Weevil','Die Back','Gall Midge','Healthy','Powdery Mildew','Sooty Mould'],
    class_mode='categorical')

Found 2800 images belonging to 8 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes = ['Anthracnose','Bacterial Canker','Cutting Weevil','Die Back','Gall Midge','Healthy','Powdery Mildew','Sooty Mould'],
    class_mode='categorical')

Found 800 images belonging to 8 classes.


In [ ]:
bottleneck_features_train = InceptionV3_model.predict_generator(train_generator, nb_train_samples)
np.save(open('bn_features_train.npy', 'wb'), bottleneck_features_train)

bottleneck_features_validation = InceptionV3_model.predict_generator(val_generator, nb_validation_samples)
np.save(open('bn_features_validation.npy', 'wb'), bottleneck_features_validation)

<ipython-input-16-78d81055e2b1>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = InceptionV3_model.predict_generator(train_generator, nb_train_samples)
<ipython-input-16-78d81055e2b1>:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = InceptionV3_model.predict_generator(val_generator, nb_validation_samples)


# 3. Создание верхней части модели. Загрузка данных в модель. Сохрание данных.

In [ ]:
train_data = np.load(open('bn_features_train.npy', 'rb'))
train_labels = np.array([0] * 350 + [1] * 350 + [2] * 350 + [3] * 350 + [4] * 350 + [5] * 350 + [6] * 350 + [7] * 350)

validation_data = np.load(open('bn_features_validation.npy', 'rb'))
validation_labels = np.array([0] * 100 + [1] * 100 + [2] * 100 + [3] * 100 + [4] * 100 + [5] * 100 + [6] * 100 + [7] * 100)

Создадание модели FFN сети и её компиляция

In [ ]:
# Создание
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(64, activation='relu')) # Первый Dense-слой
fc_model.add(Dropout(0.5, name='dropout_one')) # Первый Dropout-слой
fc_model.add(Dense(64, activation='relu')) # Второй Dense-слой
fc_model.add(Dropout(0.5, name='dropout_two')) # Второй Dropout-слой
fc_model.add(Dense(8, activation='softmax')) # Выходной слой
# Компиляция
fc_model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
fc_model.fit(train_data, train_labels,
            epochs=50, batch_size=16,
            validation_data=(validation_data, validation_labels))

fc_model.save_weights('fc_InceptionV3_mango.hdf5') # сохраняем веса

Epoch 1/50
175/175 [==============================] - 5s 16ms/step - loss: 3.0443 - accuracy: 0.1221 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 2/50
175/175 [==============================] - 1s 5ms/step - loss: 2.1056 - accuracy: 0.1193 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 3/50
175/175 [==============================] - 1s 5ms/step - loss: 2.1275 - accuracy: 0.1171 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 4/50
175/175 [==============================] - 1s 4ms/step - loss: 2.0951 - accuracy: 0.1268 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 5/50
175/175 [==============================] - 1s 5ms/step - loss: 2.0928 - accuracy: 0.1171 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 6/50
175/175 [==============================] - 1s 5ms/step - loss: 2.0853 - accuracy: 0.1100 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 7/50
175/175 [==============================] - 1s 5ms/step - loss: 2.0874 - accuracy: 0.1079 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch

In [ ]:
fc_model.evaluate(validation_data, validation_labels)

25/25 [==============================] - 0s 5ms/step - loss: 2.0794 - accuracy: 0.1250


[2.0794429779052734, 0.125]

# 4. Создание итоговой модели, загрузка в неё аугментированных данных, сохрание весов

In [ ]:
weights_filename='fc_InceptionV3_mango.hdf5'

x = Flatten()(InceptionV3_model.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
top_model=Dense(8, activation='softmax')(x)
model = Model(InceptionV3_model.input, top_model)
model.load_weights(weights_filename, by_name=True)

In [ ]:
for layer in InceptionV3_model.layers[:205]:
    layer.trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Сделаем так, чтобы в процессе обучения сохранялись только веса с наибольшей точностью на тестовой выборке:

In [ ]:
filepath="weights-improvement.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator2 = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')


Found 2800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [ ]:
  model.fit_generator(
        train_generator2,
        steps_per_epoch=nb_train_samples // 16,
        epochs=50,
        validation_data=validation_generator,
         validation_steps=nb_validation_samples // 16,
        callbacks=callbacks_list)

<ipython-input-26-7b6e6d5097aa>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/50
175/175 [==============================] - ETA: 0s - loss: 1.9950 - accuracy: 0.2186
Epoch 1: val_accuracy improved from -inf to 0.46125, saving model to weights-improvement.h5
175/175 [==============================] - 52s 256ms/step - loss: 1.9950 - accuracy: 0.2186 - val_loss: 1.5955 - val_accuracy: 0.4613
Epoch 2/50
175/175 [==============================] - ETA: 0s - loss: 1.7102 - accuracy: 0.3307
Epoch 2: val_accuracy improved from 0.46125 to 0.56875, saving model to weights-improvement.h5
175/175 [==============================] - 41s 234ms/step - loss: 1.7102 - accuracy: 0.3307 - val_loss: 1.4050 - val_accuracy: 0.5688
Epoch 3/50
175/175 [==============================] - ETA: 0s - loss: 1.5246 - accuracy: 0.4175
Epoch 3: val_accuracy improved from 0.56875 to 0.68875, saving model to weights-improvement.h5
175/175 [==============================] - 46s 265ms/step - loss: 1.5246 - accuracy: 0.4175 - val_loss: 1.1572 - val_accuracy: 0.6888
Epoch 4/50
175/175 [========

Сохранение модели в h5 формате

In [ ]:
model.save('Mango_InceptionV3.h5')

Сохранение модели в SavedModel формате

In [ ]:
model.save('Mango_InceptionV3')
path = 'Mango_InceptionV3'
file_dir = os.listdir(path)

with zipfile.ZipFile('Mango_InceptionV3.zip', mode='w', \
                     compression=zipfile.ZIP_DEFLATED) as zf:
    for file in file_dir:
        add_file = os.path.join(path, file)
        zf.write(add_file)

#5. Оценка точности модели

In [ ]:
pred_generator=test_datagen.flow_from_directory( test_dir, target_size=(224,224), batch_size=16, class_mode='categorical')
scores = model.evaluate_generator(pred_generator, 25)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Found 400 images belonging to 8 classes.


<ipython-input-29-4804fdd92db1>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(pred_generator, 25)


Аккуратность на тестовых данных: 97.00%


In [ ]:
print(model.history.history.params)

AttributeError: ignored

In [ ]:
tr_acc = model.history.history['accuracy']
tr_loss = model.history.history['loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

KeyError: ignored

Проиллюстрируем работу модели